In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import v_measure_score
from sklearn.metrics.cluster import contingency_matrix

In [ ]:
class kohonen:

    def __init__(
        self,
        M,
        N,
        X,
        learning_rate=0.1,
        neighborhood_function="gauss",
        neighborhood_proximity=1.0,
    ):
        self.M = M
        self.N = N
        self.learning_rate = learning_rate
        self.input_dim = X.shape[1]
        self.neighborhood_function = neighborhood_function
        self.neighborhood_proximity = neighborhood_proximity
        self.weights = np.random.uniform(0, 1, (self.input_dim, M, N))
        minimum = np.min(X, axis=0)
        maximum = np.max(X, axis=0)
        for i in range(self.input_dim):
            self.weights[i] = minimum[i] + (maximum[i] - minimum[i]) * self.weights[i]

    def gauss_neighborhood(self, t, T):
        return np.exp(-np.power(t * T * self.neighborhood_proximity, 2))

    def mexican_hat_neighborhood(self, t, T):
        return np.exp(-np.power(t * T * self.neighborhood_proximity, 2)) * (
            2 - 4 * np.power(t * T * self.neighborhood_proximity, 2)
        )

    def euclidean_distance(self, x, y):
        return np.linalg.norm(x - y)

    def decay(self, t, iteration_num):
        return self.learning_rate * np.exp(-t / iteration_num)

    def train(self, X, max_epochs):
        for epoch in range(max_epochs):
            X = np.random.permutation(X)
            for x in X:
                bmu = None
                bmu_distance = np.inf
                for i in range(self.M):
                    for j in range(self.N):
                        distance = self.euclidean_distance(x, self.weights[:, i, j])
                        if distance < bmu_distance:
                            bmu_distance = distance
                            bmu = (i, j)

                for i in range(self.M):
                    for j in range(self.N):
                        distance_on_lattice = self.euclidean_distance(
                            np.array(bmu), np.array([i, j])
                        )
                        if self.neighborhood_function == "gauss":
                            self.weights[:, i, j] += (
                                self.decay(epoch, max_epochs)
                                * self.gauss_neighborhood(distance_on_lattice, epoch)
                                * (x - self.weights[:, i, j])
                            )
                        elif self.neighborhood_function == "mexican_hat":
                            self.weights[:, i, j] += (
                                self.decay(epoch, max_epochs)
                                * self.mexican_hat_neighborhood(
                                    distance_on_lattice, epoch
                                )
                                * (x - self.weights[:, i, j])
                            )
                        else:
                            raise ValueError("Invalid neighborhood function")

    def predict(self, X):
        return np.array(
            [
                [
                    np.argmin(
                        [
                            self.euclidean_distance(x, self.weights[:, i, j])
                            for i in range(self.M)
                            for j in range(self.N)
                        ]
                    )
                    for x in X
                ]
            ]
        ).T